# ***IMPORT DES DONNEES DE COMPTAGES***
> il y a un exemple de verification si doublons de point dans le 17, 47, 87

In [134]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
import csv, re, os, statistics, filecmp, unidecode
import Connexion_Transfert as ct
import Import_trafics as it
import Outils
from collections import Counter
from shapely.geometry import Point,LineString, MultiLineString
from geoalchemy2 import Geometry, WKTElement
from Base_BdTopo import Import_outils as io
from Base_BdTopo import Rond_points as rp
from Base_BdTopo import Regroupement_correspondance as rc
from sqlalchemy.schema import MetaData
from sqlalchemy import inspect
from statistics import mean
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ***CD 23***
- Année 2018 : 
> à partir du fichier __Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls__<br>***attention : pour le point de comptage D941 6+152 à Aubusson, le pR est 32 et non 6. il faut donc corriger à la main le fihcier excel***
<br> Pour le moment tous les points sont déjà dans  la base, dc pas de traitement de type insert prévus.

In [ ]:
# ouvrir le classeur
df_excel=pd.read_excel(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls',skiprows=11)
# renomer les champs
df_excel_rennome=df_excel.rename(columns={'1er trimestre  du 01 janvier au 31 mars':'trim1_TV', 'Unnamed: 9':'trim1_pcpl',
                         '2ème trimestre du 01 avril au 30 juin':'trim2_TV', 'Unnamed: 11':'trim2_pcpl',
                         '3ème trimestre du 01 juillet au 30 septembre':'trim3_TV', 'Unnamed: 13':'trim3_pcpl',
                         '4ème trimestre du 01 octobre au 31 décembre':'trim4_TV', 'Unnamed: 15':'trim4_pcpl',
                         'Unnamed: 17':'pc_pl', 'TMJA 2018':'tmja'})
#supprimer la 1ere ligne
df_excel_filtre=df_excel_rennome.loc[1:,:].copy()
#mise en forme attribut
df_excel_filtre['Route']=df_excel_filtre.apply(lambda x : str(x['Route']).upper(), axis=1)
annee_cpt='2018'
#attribut id_comptag
for i in ['DEP','PR','ABS'] : 
    df_excel_filtre[i]=df_excel_filtre.apply(lambda x : str(int(x[i])),axis=1)
df_excel_filtre['id_comptag']=df_excel_filtre.apply(lambda x : '-'.join([x['DEP'],'D'+str(x['Route']),
                                                                         x['PR']+'+'+x['ABS']]),axis=1)
#donnees_mensuelles
list_id_comptag=[val for val in df_excel_filtre.id_comptag.tolist() for _ in (0, 1)]
donnees_type=['tmja','pc_pl']*len(df_excel_filtre.id_comptag.tolist())
annee_df=['2018']*2*len(df_excel_filtre.id_comptag.tolist())
janv, fev, mars,avril,mai,juin,juil,aout,sept,octo,nov,dec=[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(df_excel_filtre.id_comptag.tolist())) :
    for j in (janv, fev, mars) :
        j.extend([df_excel_filtre.trim1_TV.tolist()[i],df_excel_filtre.trim1_pcpl.tolist()[i]])
    for k in (avril,mai,juin) :
        k.extend([df_excel_filtre.trim2_TV.tolist()[i],df_excel_filtre.trim2_pcpl.tolist()[i]])
    for l in (juil,aout,sept) :
        l.extend([df_excel_filtre.trim3_TV.tolist()[i],df_excel_filtre.trim3_pcpl.tolist()[i]])
    for m in (octo,nov,dec) :
        m.extend([df_excel_filtre.trim4_TV.tolist()[i],df_excel_filtre.trim4_pcpl.tolist()[i]])
donnees_mens=pd.DataFrame({'id_comptag':list_id_comptag,'donnees_type':donnees_type,'annee':annee_df,'janv':janv,'fevr':fev,'mars':mars,'avri':avril,
              'mai':mai,'juin':juin,'juil':juil,'aout':aout,'sept':sept,'octo':octo,'nove':nov,'dece':dec})

In [ ]:
#Mise à jour bdd
with ct.ConnexionBdd('gti_otv') as c :
    c.curs.execute("select distinct id_comptag from comptage.na_2010_2018_p where dep='23' order by id_comptag")
    listerecord=[record[0] for record in c.curs]
    for id_comptag,tmja, pc_pl  in zip(df_excel_filtre.id_comptag.tolist(), df_excel_filtre.tmja.tolist(),df_excel_filtre.pc_pl.tolist()) : 
        if id_comptag in listerecord :
            c.curs.execute("update comptage.na_2010_2018_p set tmja_2018=%s, pc_pl_2018=%s, ann_cpt=%s where id_comptag=%s",(tmja, pc_pl,annee_cpt,id_comptag))
        else : 
            print (f'{id_comptag} nouveau, à traiter')
    print('fini')
    c.connexionPsy.commit()
    donnees_mens.to_sql('na_2010_2018_mensuel', c.sqlAlchemyConn,schema='comptage',if_exists='append',index=False)

# ***CD 40***
> 2019 : creation de la classe Comtage_cd40

In [ ]:
#initialiser
cd40=it.Comptage_cd40(r'D:\temp\otv\2019\Donnees_source\CD40\Observatoire des trafics')
#mettre enf orme
cd40.comptage_forme()
cd40.classer_comptage_insert_update('local_otv_station_gti', 'na_2010_2019_p', 'comptage')
#transfert donnees_agregees
cd40.update_bdd_40('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
#donnees_mensuelles
cd40.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',cd40.df_attr_mens)

# ***CD17***

> ### Année 2015 : traitée dans le fichier Import_trafics.py

> ### année 2016 données issue des borchures de comptage, uniquemnet pour ponctuels

In [ ]:
#ouverture du fichier
cpt17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_produites\donnnees_travail\Comptage\17\cpt_pctuel_but_2016.txt',
                       'brochure',2016)

#mettre à jour les id_comptage deja presents (attention, update Bdd à changé, il faut passer par creer_valeur_txt_update avant et certains paramètres ont changé)
cpt17.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2017_p', '17','ponctuel') #creer les attributs selon les donnees presentes dans la base
cpt17.update_bdd('gti_otv', 'comptage', 'na_2010_2017_p')#mise à jour

#creer referentiel si besoin
with ct.ConnexionBdd('gti_otv') as c:
    ct.ogr2ogr_shp2pg(c.connstringOgr, r'Y:\REF_GEO\BD_Topo\D17\ED16\SHP\1_DONNEES_LIVRAISON\N_TRONCON_ROUTE_BDT_017.SHP',
                     schema='referentiel',table='troncon_route_bdt17_ed16_l',geotype='MULTILINESTRING', dims=2, encodageClient='LATIN1' )
#creer graph
rqt="""
alter table referentiel.troncon_route_bdt17_ed16_l add column source integer , add column target integer ;
select pgr_createTopology ('referentiel.troncon_route_bdt17_ed16_l',1,'geom', 'ogc_fid') ;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN id TO id_ign;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN ogc_fid TO id;
alter table referentiel.troncon_route_bdt17_ed16_l add column long_km numeric ;
update referentiel.troncon_route_bdt17_ed16_l set long_km=(st_length(geom)/1000) ;
""" #attention, il manque la ligne au dessus pour créer l'analyseGraph qui va renvoyer le nb de count
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

#inserer les nouveaux comptages
with ct.ConnexionBdd('gti_otv') as c:
    cpt17.df_attr_insert.to_sql('na_2010_2017_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

#mettre à jour la geom 
rqt=""" update comptage.na_2010_2017_p
  set geom=(select geom_out  from comptage.geoloc_pt_comptag(id_comptag))
  where dep='17' and geom is null
"""
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

> ### fichier compteurs permanents format csv annee 2017 ou 2018

In [ ]:
cpt_perm=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\2018_csv_permanents_CD17.csv',
                         'permanent_csv',2018)

cpt_perm.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2018_p', '17','permanent')

#miettre à jour les données deja existantes
cpt_perm.update_bdd('gti_otv', 'comptage', 'na_2010_2018_p')#mise à jour

#inseérer les données nouvelles
cpt_perm.insert_bdd('gti_otv', 'comptage', 'na_2010_2018_p')
#mettre à jour la geom 
cpt_perm.maj_geom('gti_otv', 'comptage', 'na_2010_2018_p', '17')

# pour les données mensuelles
cpt_perm.insert_bdd_mens('gti_otv', 'comptage','na_2010_2018_mensuel')

> ### annee 2018 ; fichier compteurs tournant format excel issu des donnees pour brochure

In [ ]:
#initiliser la classe avec le fichier
bdd='gti_otv_pg11'
cpt_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\10  5 1 B3 tournants recalculés.xls',
                      'tournant_xls_bochure',2018)

#mise en forme des données
cpt_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
donnees=cpt_cd17.ouvrir_xls_tournant_brochure()
cpt_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_cd17.creer_valeur_txt_update(cpt_cd17.df_attr_update, ['id_comptag','tmja_2018','tmja_2017'])
cpt_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja_2018','tmja_2017':'tmja_2017'})

### annee 2017-2018 ; fichier ponctuel excel qui alimente des brochures

In [ ]:
#ouvrir le fichier et initialisation
cpt_pct2018_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\spécifique Brochure V85_2018 .xls',
                      'ponctuel_xls_bochure',2018)
bdd='gti_otv_pg11'
#mise en forme
cpt_pct2018_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
cpt_pct2018_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_pct2018_cd17.filtrer_periode_ponctuels_xls_brochure()
cpt_pct2018_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_pct2018_cd17.creer_valeur_txt_update(cpt_pct2018_cd17.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
cpt_pct2018_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl', 'obs_2018':'obs'})

In [ ]:
#trouver les points de comptages a inserer situe sur le même troncon elementaires que d'autres points
table_corresp=cpt_pct2018_cd17.correspondance_ancien_nouveau_comptage(bdd,'public','cd17_tournant_insert','lineaire.traf2016_bdt17_ed16_l',
                                                    'public', 'traf2016_bdt17_ed16_l','traf2016_bdt17_ed16_l_vertices_pgr')

#si le point fait partie de la table_corresp, on insère pas, sinon on insère
pt_a_inserer=cpt_pct2018_cd17.df_attr_insert.loc[~cpt_pct2018_cd17.df_attr_insert.id_comptag.isin(table_corresp.id_comptag.tolist())].copy()
#mise en form avant insertion
pt_a_inserer['dep']='17'
pt_a_inserer['route']=pt_a_inserer.id_comptag.apply(lambda x : x.split('-')[1])
pt_a_inserer.rename(columns={'absc':'abs','tmja':'tmja_2018','pc_pl':'pc_pl_2018','obs':'obs_2018'},inplace=True)
pt_a_inserer['reseau']='RD'
pt_a_inserer['gestionnai']='CD17'
pt_a_inserer['concession']='N'
pt_a_inserer['type_poste']='ponctuel'
pt_a_inserer=pt_a_inserer[['id_comptag','dep','route','pr','abs','reseau','gestionnai','concession','type_poste','tmja_2018','pc_pl_2018','obs_2018']].copy()
#si plusieurs fois le mm point on garde la valeur max
pt_a_inserer=pt_a_inserer.loc[pt_a_inserer.tmja_2018==pt_a_inserer.groupby('id_comptag').tmja_2018.transform(max)].copy()

cpt_pct2018_cd17.insert_bdd(bdd, 'comptage', 'na_2010_2018_p', pt_a_inserer)
cpt_pct2018_cd17.maj_geom(bdd, 'comptage', 'na_2010_2018_p', dep='17')

# ***CD19***
> 2019 : creation de la classe Comptage_cd19

In [ ]:
cd19=it.Comptage_cd19(r'D:\temp\otv\2019\Donnees_source\CD19\2019_Recensement_trafic.xls',2019)
cd19.comptage_forme()

In [ ]:
#correspondance existant
cd19.classer_comptage_update_insert(self,'local_otv_station_gti','na_2010_2019_p','comptage',
                                       'local_otv_station_gti', 'public','cd19_cpt','referentiel.troncon_route_bdt19_ed17_l',
                                        'referentiel', 'troncon_route_bdt19_ed17_l','troncon_route_bdt19_ed17_l_vertices_pgr','id')

In [ ]:
#maj existant
cd19.update_bdd_19('local_otv_station_gti','comptage','na_2010_2019_p')

# ***CD47***
> Annee des trafics 2018, fichiers : Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018
les fichiers sont decomposes en permanents tournants temporaires, il faut recomposer les donnees

In [ ]:
#initialiser un objet
cpt47=it.Comptage_cd47(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018','TRAFICS PERMANENTS')
#calculer les attributs de comptages : df_attr, df_attr_insert et df_attr_update, en prenant en compte les comptages existants
cpt47.classer_comptage_update_insert('gti_otv_pg11')

In [ ]:
#mise à jour des données déjà présentes dansla base
val_txt=cpt47.creer_valeur_txt_update(cpt47.df_attr_update, ['id_comptag','tmja','pc_pl'])
cpt47.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl'})

In [ ]:
#traitement des données non présentes (df_attr_insert)
#recherche de correspondance pour les permanents et tournants
dico_corresp=cpt47.corresp_old_new_comptag('gti_otv_pg11', 'public','cpt47_temp','lineaire.traf2017_bdt47_ed17_l',
                            'referentiel', 'troncon_route_bdt47_ed17_l','troncon_route_bdt47_ed17_l_vertices_pgr','id')

In [ ]:
#filtre des données df_attr_insert selon l'id_comptag present dans le dico_corresp
cpt47.df_attr_insert=cpt47.df_attr_insert.loc[~cpt47.df_attr_insert.id_comptag.isin(df_correspondance.id_comptag.to_list())].copy()
# POUR INFO, l'id_comptag présent dan sle dico_corresp a été transférer à la main dans l table comptage.corresp_id_comptag
#mettre en forme les attributs avant insert
cpt47.mise_en_forme_insert('2018')
#inserer les donnes dans la table
cpt47.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', cpt47.df_attr_insert)

In [ ]:
#mettre à jour la géométrie
cpt47.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', '47')

> ## **2019**

In [443]:
cpt47=it.Comptage_cd47(r'D:\temp\otv\2019\Donnees_source\CD47','TRAFICS PERIODIQUES',2019)
#calculer les attributs de comptages : df_attr, df_attr_insert et df_attr_update, en prenant en compte les comptages existants, mais ans recherche des équivalences avec les anciens comptage snon recensées
cpt47.classer_comptage_update_insert('local_otv_station_gti')

In [344]:
#qq points tournants non connus
cpt47.df_attr_insert.loc[cpt47.df_attr_insert['type_poste'].isin(['permanent','tournant'])]
#filtrer les 2 points bizarres
cpt47.df_attr=cpt47.df_attr.loc[cpt47.df_attr.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()
cpt47.df_attr_insert=cpt47.df_attr_insert.loc[cpt47.df_attr_insert.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()
cpt47.df_attr_update=cpt47.df_attr_update.loc[cpt47.df_attr_update.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()

In [345]:
#obtention dico de correspondance
dico_corresp=cpt47.corresp_old_new_comptag('local_otv_station_gti', 'public','cpt47_temp','referentiel.troncon_route_bdt47_ed17_l',
                            'referentiel', 'troncon_route_bdt47_ed17_l','troncon_route_bdt47_ed17_l_vertices_pgr','id')
#insertion dans bdd
cpt47.insert_bdd('local_otv_station_gti','comptage','corresp_id_comptag',dico_corresp[['id_comptag','id_comptag_lin']].rename(columns={'id_comptag':'id_gest','id_comptag_lin':'id_gti'}))
#calcul nouveau id_comptag
cpt47.comptag_existant_bdd('local_otv_station_gti',schema='comptage', table='na_2010_2019_p',dep='47', type_poste=False)
cpt47.corresp_nom_id_comptag('local_otv_station_gti',cpt47.df_attr)
cpt47.df_attr_update=cpt47.df_attr.loc[cpt47.df_attr.id_comptag.isin(cpt47.existant.id_comptag.tolist())].copy()
cpt47.df_attr_insert=cpt47.df_attr.loc[~cpt47.df_attr.id_comptag.isin(cpt47.existant.id_comptag.tolist())].copy()
#mise a jour
cpt47.update_bdd_47('local_otv_station_gti','comptage', 'na_2010_2019_p',2019,cpt47.df_attr_update)

In [363]:
#donnees a inserer
cpt47.mise_en_forme_insert()
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_p', cpt47.df_attr_insert)
cpt47.maj_geom('local_otv_station_gti','comptage', 'na_2010_2019_p', '47')
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_mensuel', cpt47.df_attr_mens)
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_horaire', cpt47.df_attr_horaire)

# ***CD87***
> Dans ce Departement les donnees sont fournies en fichiers .fim. il vaut recalculer les valeusr de comptages <br> Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD87\Fichiers FIM 87 -2018

### ***lister les voies, classer les types de commptages, lister les fichiers a regrouper***
> le dossier contient un tres grand nombre de fihciers, parfois ils sont a regrouper, parfois la structure de nommage varie

In [ ]:
d87=it.Comptage_cd87(r'D:\temp\otv\87\Fichiers FIM 87 -2018') 
d87.dico_pt_cptg() #creer le dico de base

In [ ]:
d87.dico_voie

### ***Traiter les donnees***
> Pour chaque route référencée dans le dico, on va calculer les TMJA %PL, date_debut, date_fin, type_poste de chaque fichiers puis
faire les calculs si necessaires (moyenne si plsr fichiers).<br> ensuite on classe puis update et insert

In [ ]:
#mettre à jour le dico et le transformer en dataframe sans les ponctuels pendant les grandes vacances
d87.dataframe_dico_glob()
#preparer les données
d87.classer_comptage_update_insert('gti_otv_pg11','na_2010_2018_p','comptage'
                                   'public','d87_cpt_temp','lineaire.traf2017_bdt87_ed17_l',
                                  'referentiel', 'troncon_route_bdt87_ed17_l','troncon_route_bdt87_ed17_l_vertices_pgr','id')
#mettre à jour les données
d87.update_bdd_d87('gti_otv_pg11','na_2010_2018_p','comptage')
#insérer les données et mettre à jour les geom
d87.insert_bdd_d87('gti_otv_pg11','na_2010_2018_p','comptage')

# ***CD 16***
- Année 2018 : 
> à partir du fichier D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx on traite les permanents.<br> à partir des donnees sur PIGMA on traite les compteurs temporaires <br>***attention : il y a aussi des données de comptages temporaires en FIM qui permettront d'obtenir de la données horaires***

In [ ]:
#2018
cd16=it.Comptage_cd16(r'D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx',r'D:\temp\otv\Donnees_source\CD16\pigma\comptages_routiers_2019\comptages_routiers.shp',
                      r'D:\temp\otv\Donnees_source\CD16\pigma\position_compteurs_2019\position_compteurs.shp',2018)
cd16.comptage_forme()
cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.update_bdd_16('local_otv','comptage','na_2010_2018_p')

In [ ]:
#2019
cd16=it.Comptage_cd16(r'D:\temp\otv\2019\Donnees_source\CD16\B15_2019.xlsx',r'D:\temp\otv\2019\Donnees_source\CD16\SIG_Comptages_CD16\Comptages_routiers.shp',
                      r'D:\temp\otv\2019\Donnees_source\CD16\SIG_Comptages_CD16\Position_compteurs.shp',2019)
cpt_a_ignorer=('16-D731-27+45','16-D910-23+821', '16-D1000-16+35')
cd16.comptage_forme(7,('16-D731-27+45','16-D910-23+821', '16-D1000-16+35'),r'D:\temp\otv\2019\Donnees_source\CD16\Comptages_secondaires_CD16_2019')
#cd16.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')
#cd16.update_bdd_16('local_otv_station_gti','comptage', 'na_2010_2019_p')
#cd16.insert_bdd('local_otv_station_gti','comptage','na_2010_2019_mensuel',cd16.df_attr_mens)
#cd16.insert_bdd('local_otv_station_gti','comptage','na_2010_2019_horaire',cd16.df_attr_horaire)

# ***CD 86***
Attention , dans ce département parfois ce sont les données en PLOD et ASCD qui sont ok, parfois c'est le cumulD <br>
Dans ce Dept, les Compmtages dites 'Secondaires' sont équivalent à 'tournants', et 'Tournants' equivalent à 'ponctuel'
- Année 2018 : 
> à partir des fichier D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx et  D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls on traite les permanents et secondaires <br> il y a un petit pb sur les compteurs permanents entre la donnees pr+abs chez nous et la leur dans le tableau, donc il faut la premiere fois tout passer dans la table de correspondance.<br> dans ce dept pas de nouveau points
- Année 2019 : 
> un seul fichier avec les comptages sur 2feuilles : permanents et tournants.<br> la feuille permanent contient aussi des données 'Tournant' qui sont masquées, pour lesquelles les reference en abscisse sont tuojours à 0<br> 3 nouveaux points tournants ajoutés à  la main, le reste des ponctuels en auto

In [ ]:
#2018
cd86=it.Comptage_cd86(r'D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx',r'D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls')

#si besoin de dico corresp : 
corr=cd86.corresp_perm('local_otv', 'na_2010_2018_p')
cd86.insert_bdd('local_otv', 'comptage','corresp_id_comptag',corr)

#sinon
cd86.comptage_forme()
cd86.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')
cd86.update_bdd_86('local_otv','comptage','na_2010_2018_p')

In [563]:
#2019
cd86=it.Comptage_cd86(r'D:\temp\otv\2019\Donnees_source\CD86\Comptages globaux 2019 tournants et permanents.xlsx',r'D:\temp\otv\2019\Donnees_source\CD86\Comptages globaux 2019 tournants et permanents.xlsx',2019,
                     'permanents','secondaires')
cd86.comptage_forme('local_otv_station_gti', 'na_2010_2019_p')
cd86.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')
cd86.update_bdd_86('local_otv_station_gti','comptage','na_2010_2019_p')

In [544]:
cd86.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')

In [561]:
cd86.df_attr_insert['dep']='86'
cd86.df_attr_insert['reseau']='RD'
cd86.df_attr_insert['gestionnai']='CD86'
cd86.df_attr_insert['concession']='N'
cd86.df_attr_insert['obs']="nouveau_point 2019, denomination CD86='tournant'"
cd86.df_attr_insert.rename(columns={'absc' : 'abs', 'tmja':'tmja_'+str(cd86.annee),'pc_pl':'pc_pl_'+str(cd86.annee),'obs':'obs_'+str(cd86.annee), 'src':'src_'+str(cd86.annee)},inplace=True)

In [567]:
cd86.insert_bdd_86('local_otv_station_gti', 'comptage','na_2010_2019_p')

# ***CD 24***
- Année 2018 : 
> à partir des fichier D:\temp\otv\Donnees_source\CD24\2018_CD24_trafic.csv <br> il n'y a que les compteurs permanents.<br> **Données Mensuelles dispos**.<br> Pensez que des céhanges ont ue lieu pour récupérer tous les points de comptages (perm, tourn, ponct). <br> ***Gros pb de geooloc des PR ign***

In [ ]:
cd24=it.Comptage_cd24(r'D:\temp\otv\Donnees_source\CD24\2018_CD24_trafic.csv')
cd24.comptage_forme()
cd24.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2018_p')

In [ ]:
cd24.update_bdd_24('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

In [ ]:
cd24.insert_bdd_24('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

In [ ]:
cd24.df_attr_insert

# ***SCA***
- Année 2018 : 
> Concerne COFIROUTE, VINCI, ALIENOR, ATLANDES.<br> Des données de correspondances id_comptages sont dans la table source de la base de données otv.<br> **Donnees TMJM dispos**

- ### COFIROUTE 
> Que 5 points, faits à la main

- ### VINCI 
> L'idée c'est d'importer le tableur et les données et de faire la jointure pour mise à jour.<br>**ATTENTION : rien de prévu si nouveau points** 

In [ ]:
#2018
cpt=it.Comptage_vinci(r'D:\temp\otv\Donnees_source\VINCI\2018_comptage_vitesse_moyenne_VINCI.xlsx')
cpt.update_bdd_Vinci('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

In [58]:
#2019
cpt=it.Comptage_vinci(r'D:\temp\otv\2019\Donnees_source\ASF\bdd_point_comptage et vitesse moyenne 2019.xlsx', 2019)
cpt.update_bdd_Vinci('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
cpt.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',cpt.df_attr_mens)

- ### ALIENOR / ATLANDES
> Que 7 points chaucun, j'utilise surtout là pour les donees mensuelles

In [108]:
alienor=it.Comptage_alienor(r'D:\temp\otv\2019\Donnees_source\ALIENOR\ALIENOR_trafic_2019.xlsx',2019)
alienor.ouvrir_et_separe_donnees()
alienor.update_bdd_Alienor('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
alienor.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',alienor.df_attr_mens)

In [204]:
atlandes=it.Comptage_atlandes(r'D:\temp\otv\2019\Donnees_source\ATLANDES\2019_trafic_atlandes_7_boucles.xls',2019)
atlandes.donnees_mens()
alienor.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',atlandes.df_attr_mens)

# ***Grand Poitiers***
> on a deja un fichier dans la base de donnees : on va chercher a greffer les donnees et a cree un nouvel id_comptag

In [359]:
#ouvrir un fichier
dossier=r'D:\temp\otv\2019\Donnees_source\GP\Automatiques 2019'
cpt=it.Comptage_GrandPoitiers(r'D:\temp\otv\2019\Donnees_source\GP\Automatiques 2019',2019)
cpt.comptage_forme('local_otv_station_gti')
cpt.update_bdd_grdPoi('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
cpt.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_horaire', cpt.df_attr_horaire)

# ***Ville Anglet***
> Ci-dessous simplement un exemple d'ouverture de fcihier .mdb, de liste des tables et de lecture d'une table

In [ ]:
with ct.ConnexionBdd('mdb', fichierMdb=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\Anglet_Commune\COMPTAGES ANGLET\Comptages 2019\donnees brutes\1.Av .des Dauphins.mdb') as c:
    tables = list(c.mdbCurs.tables())
    query = "SELECT * FROM A1"
    df = pd.read_sql(query, c.connexionMdb)

In [ ]:
tables

In [ ]:
df.head(1)

# ***TESTS***

In [ ]:
## DETAILS OPERATIONS CD87

#regrouper les différentes données issues de ficiers et les ajouter au dico
for k, v in d87.dico_voie.items() : 
    for i,e in enumerate(v) : 
        if len(e['fichiers'])==1 : 
            print(e['fichiers'][0])
            obj_fim=it.FIM(os.path.join(d87.dossier,e['fichiers'][0]))
            try : 
                obj_fim.resume_indicateurs()
            except obj_fim.fim_PasAssezMesureError : 
                continue
            except Exception as ex : 
                print(f"erreur : {ex} \n dans fichier : {e['fichiers'][0]}")
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=obj_fim.tmja, obj_fim.pc_pl, obj_fim.date_debut,obj_fim.date_fin
        elif len(e['fichiers'])>1 :
            list_tmja=[]
            list_pc_pl=[]
            for f in e['fichiers'] : 
                obj_fim=it.FIM(os.path.join(d87.dossier,f))
                print(f)
                try : 
                    obj_fim.resume_indicateurs()
                except (obj_fim.fim_PasAssezMesureError,obj_fim.fimNbBlocDonneesError)  : 
                    continue
                except Exception as ex : 
                    print(f"erreur : {ex} \n dans fichier : {f}")
                list_tmja.append(obj_fim.tmja)
                list_pc_pl.append(obj_fim.pc_pl)
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=int(mean(list_tmja)), round(mean(list_pc_pl),2),np.NaN, np.NaN

#renseigner le type de poste
for k, v in d87.dico_voie.items() : 
    for e in v : 
        if len(e['fichiers']) > 4 :
            e['type_poste']='permanent'
        elif 1<len(e['fichiers'])<=4 : 
            e['type_poste']='tournant'
        elif len(e['fichiers'])== 1 :
            e['type_poste']='ponctuel'
        else : 
            e['type_poste']='NC'

#faire une df avec les points de comptage
d87.df_attr=pd.DataFrame([[k, e['pr'], e['abs'], e['tmja'], e['pc_pl'], e['type_poste'],
                      e['date_debut'],e['date_fin']] for k, v in d87.dico_voie.items() for e in v if 'tmja' in e.keys()], 
             columns=['route','pr','absc','tmja','pc_pl','type_poste','date_debut','date_fin'])
d87.df_attr['id_comptag']=d87.df_attr.apply(lambda x :'87-'+x['route']+'-'+str(x['pr'])+'+'+str(x['absc']), axis=1)

#filtre cpt ponctuel
d87.df_attr=d87.df_attr.loc[d87.df_attr.apply(lambda x : x['date_debut'].month not in [7,8] and x['date_fin'].month not in [7,8], 
                                                         axis=1)].copy()

#fare le tri avec les comptages existants : 
#recuperer les compmtages existants
d87.comptag_existant_bdd('gti_otv_pg11', 'na_2010_2018_p', schema='comptage',dep='87', type_poste=False)
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
#obtenir une cle de correspondace pour les comptages tournants et permanents
df_correspondance=d87.corresp_old_new_comptag('gti_otv_pg11', 'public','d87_cpt_temp','lineaire.traf2017_bdt87_ed17_l',
                                'referentiel', 'troncon_route_bdt87_ed17_l','troncon_route_bdt87_ed17_l_vertices_pgr','id')

#passer la df de correspondance dans le table corresp_id_comptage
d87.insert_bdd('gti_otv_pg11', 'comptage', 'corresp_id_comptag', 
               df_correspondance.rename(columns={'id_comptag_lin':'id_gti','id_comptag':'id_gest'})[['id_gest','id_gti']])

#faire la correspondance entre les noms de comptage
d87.corresp_nom_id_comptag('gti_otv_pg11',d87.df_attr)

#recalculer les insert et update
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()

#mettre en forme pour update
d87.df_attr_update['obs']=d87.df_attr_update.apply(lambda x : x['date_debut'].strftime('%d/%m/%Y')+'-'+ x['date_fin'].strftime('%d/%m/%Y') if not pd.isnull(x['date_debut']) else '', axis=1)
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'obs']='pc_pl inconnu'
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'pc_pl']=-99

#preparer update
valeurs_txt=d87.creer_valeur_txt_update(d87.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
dico_attr={'tmja_2018':'tmja','pc_pl_2018':'pc_pl','obs_2018':'obs'}
#update
d87.update_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', valeurs_txt,dico_attr)

#mettre en forme le insert
dbl=d87.df_attr_insert.loc[d87.df_attr_insert.duplicated('id_comptag', False)].copy()
ss_dbl=d87.df_attr_insert.loc[~d87.df_attr_insert.index.isin(dbl.index.tolist())].copy()
dbl=dbl.dropna()
dbl_traite=dbl.loc[dbl.tmja==dbl.groupby('id_comptag').tmja.transform(max)].drop_duplicates().copy()
d87.df_attr_insert=pd.concat([dbl_traite,ss_dbl], axis=0, sort=False)
d87.df_attr_insert.pc_pl.fillna(-99, inplace=True)
annee='2018'
d87.df_attr_insert['dep']='87'
d87.df_attr_insert['reseau']='RD'
d87.df_attr_insert['gestionnai']='CD87'
d87.df_attr_insert['concession']='N'
d87.df_attr_insert['obs']=d87.df_attr_insert.apply(lambda x : f"""nouveau_point,{x['date_debut'].strftime("%d/%m/%Y")}-{x['date_fin'].strftime("%d/%m/%Y")}""" if not (pd.isnull(x['date_debut']) and  pd.isnull(x['date_fin'])) else None,axis=1)
d87.df_attr_insert.rename(columns={'absc' : 'abs', 'tmja':'tmja_'+annee,'pc_pl':'pc_pl_'+annee,'obs':'obs_'+annee},inplace=True)
d87.df_attr_insert.drop(['date_debut','date_fin','route'],axis=1,inplace=True)

#mettre à jour la geom
d87.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', dep='87')

In [5]:
with ct.ConnexionBdd('local_otv_station_gti') as c : 
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'D:\temp\otv\2019\donnees_produite\na_2010_2019_p_dira\na_2010_2019_p_dira.shp',schema='public', table='na_2010_2019_p_dira',SRID='2154',geotype='POINT', dims=2, creationMode='',encodageClient='UTF-8', requeteSql='')

debut import fichier D:\temp\otv\2019\donnees_produite\na_2010_2019_p_dira\na_2010_2019_p_dira.shp avec shape2pg à 15:54:09 
 avec commande ogr2ogr  -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt POINT -dim 2 -lco "SCHEMA=public" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432" D:\temp\otv\2019\donnees_produite\na_2010_2019_p_dira\na_2010_2019_p_dira.shp -nln public.na_2010_2019_p_dira 
Fait
